<a href="https://colab.research.google.com/github/m-soro/store_files/blob/main/PaperStore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown ## Create Paper Supply Template 

#@markdown <img src='https://ideal-ecopack.com/wp-content/uploads/2019/08/Food_Service-300x300.jpg'>
 
#@markdown > *You must be signed in to your google account to use this app*

#@markdown 1. **Start** here, click `execute` button <img src='https://github.com/m-soro/store_files/blob/main/execute.png?raw=true'> **TWICE**

#@markdown * **First** click will import neccessary packages: 
#@markdown > <img src="https://github.com/m-soro/store_files/blob/main/first.png?raw=true" alt="install confirmation" width="500" height="90">

#@markdown * **Second** click, none will be returned. **Okay to proceed**.

#@markdown ---

#@markdown > *If a **warning prompt** pops-up, click `run anyway`*

#@markdown > <img src="https://github.com/m-soro/store_files/blob/main/warn.png?raw=true" alt="warning" width="500" height="200"> 

try:
    import pandas as pd
    import urllib.request
    import xlsxwriter
    from google.colab import files
    import datetime
    import ipywidgets as widgets
except:
    !pip install xlsxwriter

# Get csv file from github
url = 'https://raw.githubusercontent.com/m-soro/store_files/main/paper.csv'
urllib.request.urlretrieve(url, 'paper.csv')
df = pd.read_csv("paper.csv",encoding='latin-1') 
df = df[6:-4]

df.dropna(subset = ["Unnamed: 1"], inplace=True)
df = df[df["Outlet Inventory Sheet"].str.contains("Category")==False]

df.drop(columns=['Unnamed: 2', 'Unnamed: 4', 'Unnamed: 6', 'Unnamed: 7'], inplace=True)
df.columns = ['Item_Num', 'Desc', 'Price','Unit']

df['Desc'] = [n.replace('-',' ') for n in df.Desc] 

selected_data = []

In [ ]:
#@title Search by Item Description { run: "auto", vertical-output: true }

#@markdown 
#@markdown  2. Enter item Description below and click `execute` button
#@markdown  * Check the items you wish to add your template.
#@markdown  * Click `Add to Template` to include in the template.
#@markdown  * Repeat above steps to add more items. 
#@markdown
#@markdown  > If left **blank**, the **whole paper storeroom catalog** will be returned.

#@title  { run: "auto" }
Item = "" #@param {type:"string"}
filteredDF = df[df['Desc'].str.contains((Item).title())]

checkboxes = [widgets.Checkbox(layout=widgets.Layout(width='150%'),value=False, description=str(label)) for label in filteredDF.Desc]

output = widgets.VBox(children=checkboxes)

display(output)

In [ ]:
#@title Add to Template { run: "auto" }
# @markdown This will return what is added to template 
# @markdown >Uncheck the item after adding to avoid duplication
for i in range(0, len(checkboxes)):
    if checkboxes[i].value == True:
        selected_data.append(checkboxes[i].description.split('-')[0])

added_to_template = pd.DataFrame(selected_data, columns=['Items'])
added_to_template

In [ ]:
#@title Clear Template
#@markdown This will **delete all** selected items from the template
selected_data.clear()


In [ ]:
#@title Build/Save/Download the Template

#@markdown  3. Enter the name of your template
#@markdown  * A printable template will auto-download including the creation date


template_name = "paper template name" #@param {type:"string"}
template = pd.DataFrame(selected_data)

sel_item_num = [df.query('Desc == @n').Item_Num.values[0] for n in template[0]]
sel_desc = [df.query('Desc == @n').Desc.values[0] for n in template[0]]
sel_unit = [df.query('Desc == @n').Unit.values[0] for n in template[0]]
    
template = pd.DataFrame(sel_item_num)
template['Desc'] = sel_desc
template['Unit'] = sel_unit
template['Order_Qty'] = ' '
template.columns = ['Item_Num', 'Description','Unit', 'Order Qty']

sheet_name = template_name
d = datetime.datetime.now().strftime("%m.%d.%y")
writer = pd.ExcelWriter(sheet_name + '_' + d +'.xlsx',engine='xlsxwriter')
template.to_excel(writer,sheet_name=sheet_name,index=False)
worksheet = writer.sheets[sheet_name]
worksheet.set_column('B:B', template['Description'].str.len().max())
worksheet.set_header(sheet_name + '_' + d)
worksheet.fit_to_pages(1, 1)
writer.save()
files.download(sheet_name + '_' + d +'.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>